In [5]:
import scipy.io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageDraw

In [151]:
mat = scipy.io.loadmat(r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\egohand\metadata.mat')

In [281]:
def save_mask_egohand():
    # 데이터셋 전체에 대한 메타 데이터
    mat = scipy.io.loadmat(r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\egohand\metadata.mat')

    # 전체 디렉토리    
    for direc in range(48):
        base_path = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\egohand\_LABELLED_SAMPLES'
        d_name = mat['video'][:, direc][0][0][0]
        dir_path = os.path.join(base_path, d_name)

        # 디렉토리 하나
        for mask in mat['video'][:, direc][0][6]:
            
            # 이미지 하나
            for frame, *polys in mask:
                # 마스크 초기화
                mask = np.zeros(shape=(720, 1280), dtype=np.uint8)

                # 유효한 폴리곤 추출
                polys = [poly.astype(np.int32) for poly in polys if len(poly) > 3]
                
                # 마스크에 폴리곤 데이터 씌워서 mask 로 만들기
                cv2.fillPoly(mask, polys, 1)

                # 저장할 경로
                img_path = os.path.join(dir_path, f"frame_{('0'*4+str(frame[0][0]))[-4:]}.jpg")
                save_path = os.path.join(dir_path, f"mask_{('0'*4+str(frame[0][0]))[-4:]}.jpg")
                to_save = Image.fromarray(mask*255)
                to_save.save(save_path)
                ########### viz
                # contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                # cv2.drawContours(mask, contours[0], -1, (255, 0, 0), 2)

                # plt.imshow(mask)
                # img = Image.open(img_path)
                # draw = ImageDraw.Draw(img)
                # for contour in contours[0]:
                #     draw.polygon(list(contour.reshape(1, -1)[0]), width=3)
                # img

In [13]:
from glob import glob
import pandas as pd

def make_lut():
    img_pathes = list()
    mask_pathes = list()

    # 경로 확인
    for dir_path in glob(r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\egohand\_LABELLED_SAMPLES\*'):
        for p in glob(f'{dir_path}/*'):
            if 'frame' in p:
                img_pathes.append(p)
                mask_pathes.append(p.replace('frame', 'mask'))

    df = pd.DataFrame()
    df['img'] = img_pathes
    df['mask'] = mask_pathes

    # csv 저장
    df.to_csv('lut.csv')

In [282]:
save_mask_egohand()

In [15]:
make_lut()